In [1]:
import pandas as pd
import seaborn as sns
import plotly as pt
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.io as pio
import folium as fl
import nbformat as nb
import ipywidgets as wi
from collections import Counter
from IPython.display import display
import Biblioteca_Proyecto as bp
import os
import json

In [2]:

data_list = []

for i in os.listdir("C:\\Users\\User\\OneDrive\\Escritorio\\RESTAURANTES_PROYECTO01\\Json_Data"):
    if i.endswith(".json"):
        with open(os.path.join("C:\\Users\\User\\OneDrive\\Escritorio\\RESTAURANTES_PROYECTO01\\Json_Data", i), "r", encoding="utf-8") as file:
            json_loaded = json.load(file)
            data_list.append(json_loaded)

data = pd.DataFrame(data_list)
data2 = pd.json_normalize(data_list)

Ya Alejandro lleva 3 anos en los Estados Unidos y sus amigos le aconsejaron que lo mejor opcion para ayudar a su familia en Cuba era poniendoles un negocio, el estaba decidido a invertir en algo que no solo generara ingresos, sino que tambien mejorara la calidad de vida de sus padres en La Habana. Muchos le sugirieron algunas de las opciones mas populares en Cuba, como revender productos Shein, sin embargo el tenia una vision diferente,  su objetivo era establecer un negocio autosuficiente, y sabiendo ya que no habia mujer en La Habana que cocinara mejor que su vieja decidio que abrir un restaurante era la mejor opcion para lograr su objetivo....
Su madre Caridad una ama de casa que dedico toda su vida a su esposo, a su hijo y a la cocina casi infarata al escuchar la noticia de que por finnnn le van a empezar a pagar por cocinar algo que llevaba toda la vida haciendo por amor y sin ningun tipo de lucro.

Para que todo saliera bien, Alejandro sabia que necesitaba resalizar un analisis exhaustivo de la competencia, por lo que tomo una muestra de los restaurantes de toda La Habana...



In [3]:
#Tabla con la cantidad de jsons que hay de cada municipio :)
municipio = data2["location.municipe"].str.lower().str.strip()

restaurantes_municipio = municipio.value_counts().reset_index()
restaurantes_municipio.columns = ["municipio", "cantidad_restaurantes"]


fig = go.Figure(data=[go.Table(
    header=dict(values=["Municipio", "Cantidad de Restaurantes"],
                fill_color="green",
                align="left"),
    cells=dict(values=[restaurantes_municipio["municipio"], restaurantes_municipio["cantidad_restaurantes"]],
               fill_color="lavender",
               align="left"))
])


fig.update_layout(title="Cantidad de Restaurantes por Municipio")
fig.show()

total = restaurantes_municipio['cantidad_restaurantes'].sum()
print("Total de Restaurantes: ",total)


Total de Restaurantes:  149


Lo primero era estudiar el mercado para conocer los precios de la competencia, el queria que sus precios estuvieran por debajo de una gran parte de los restaurantes de glamur, que como le decia su madre: "mijo ese lugar es solo para turistas", porque siempre considero los precios "locos" un abuso para el cubano de a pie y queria que su restaurante aunque no tuviera mucho lujo fuera algo absequible al menos para la mayoria.

In [4]:
data2["menu_price"] = data2.apply(bp.c_precio_medio, axis=1)

data3 = data2.dropna(subset=["menu_price"])

data3 = data2.sort_values(by=["location.municipe", "menu_price"], ascending=[True, True])

municipios = data2["location.municipe"].unique()
info_municipio = {municipio: data3[data3["location.municipe"] == municipio] for municipio in municipios}

def mostrar_tabla(municipio):
    display(info_municipio[municipio][["name", "menu_price"]])

selector_municipio = wi.Dropdown(options=municipios, description="Municipio:")
wi.interact(mostrar_tabla, municipio=selector_municipio)



interactive(children=(Dropdown(description='Municipio:', options=('Marianao', 'Habana del Este', 'Habana Vieja…

<function __main__.mostrar_tabla(municipio)>

Despues que quedo comprobado que la media de los precios de los diferentes citios iban desde los 400 cup en pequenos cafes, o cafeterias estatales, hasta 4000 cup en restaurantes mas refiinados en el corazon del Vedado o en las historicas plasas de la Habana Vieja, decidio dejar que el suyo tendria un media de 1000 cup ya conocia el dato de que el salario medio de los cubanos es de 4.648 pesos cubanos segun la Oficina Nacional de Estadistica e Informacion (ONEI) y dado que el precio medio de su restaurante vendria siendo el 21.51% de del sueldo no le parecia lo mas correcto pero no podia hacer nada mas debido al aumento de los productos que necesitaria para elaborar su menu.





Ay!!, mijo lo unico que te va a pedir tu madre es que por favor no me pongas a preparar moderneses tu sabes que mi cocina es a la antigua y compra sazones naturales que para ti no es novedad el odio que le tengo a cocinar con polvitos de esos.
Alejandro: Tranquila vieja aqui se cocina lo que tu digas y tu sabes que no hay cosa que nos guste mas a nosotros que los platillos mas comunes, aqui no somos de cabiar, ni de salmon, hasta el restaurante mas glamuroso de La Habana tiene que tener en su menu una oferta de ropa vieja y unas buenas croquetas.

In [5]:
#Grafico de barras que muestra cuales son los 3 (entrantes,principales,pizzas,postres,cocteles) mas comunes en los menu :)

entrantes_nombres = []
entrantes_comunes = []

for index, row in data.iterrows():
    menu = row["menu"]
    if isinstance(menu, dict) and "starters" in menu:  
        for starter in menu["starters"]:
            nombre = starter.get("item_name")
            if nombre is not None: 
                nombre_igualado =bp.i_nombre(nombre)
                entrantes_nombres.append(nombre_igualado)
counter = Counter(entrantes_nombres)
tres_mas_comunes = counter.most_common(3)
for nombre, frecuencia in tres_mas_comunes:
    entrantes_comunes.append((nombre, frecuencia))
    

principales_nombres = []
principales_comunes = []

for index, row in data.iterrows():
    menu = row["menu"]
    if isinstance(menu, dict) and "mains" in menu:  
        for starter in menu["mains"]:
            nombre = starter.get("item_name")
            if nombre is not None: 
                nombre_igualado = bp.i_nombre(nombre)
                principales_nombres.append(nombre_igualado)
counter = Counter(principales_nombres)
tres_mas_comunes = counter.most_common(3)
for nombre, frecuencia in tres_mas_comunes:
    principales_comunes.append((nombre, frecuencia))

pizza_nombres = []
pizza_comunes = []

for index, row in data.iterrows():
    menu = row["menu"]
    if isinstance(menu, dict) and "pizza" in menu:  
        for starter in menu["pizza"]:
            nombre = starter.get("item_name")
            if nombre is not None: 
                nombre_igualado = bp.i_nombre(nombre)
                pizza_nombres.append(nombre_igualado)
counter = Counter(pizza_nombres)
tres_mas_comunes = counter.most_common(3)
for nombre, frecuencia in tres_mas_comunes:
    pizza_comunes.append((nombre, frecuencia))

postres_nombres = []
postres_comunes = []

for index, row in data.iterrows():
    menu = row["menu"]
    if isinstance(menu, dict) and "desserts" in menu:  
        for starter in menu["desserts"]:
            nombre = starter.get("item_name")
            if nombre is not None: 
                nombre_igualado = bp.i_nombre(nombre)
                postres_nombres.append(nombre_igualado)
counter = Counter(postres_nombres)
tres_mas_comunes = counter.most_common(3)
for nombre, frecuencia in tres_mas_comunes:
    postres_comunes.append((nombre, frecuencia))


cocteles_nombres = []
cocteles_comunes = []
for index, row in data.iterrows():
    menu = row["menu"]
    if isinstance(menu, dict) and "cocktails" in menu:  
        for cocktail in menu["cocktails"]:
            nombre = cocktail.get("item_name")
            if nombre is not None: 
                nombre_igualado = bp.i_nombre(nombre)
                cocteles_nombres.append(nombre_igualado)
counter = Counter(cocteles_nombres)
tres_mas_comunes = counter.most_common(3)
for nombre, frecuencia in tres_mas_comunes:
    cocteles_comunes.append((nombre, frecuencia))

starters = entrantes_comunes
mains = principales_comunes
pizzas = pizza_comunes
desserts = postres_comunes
cocktails = cocteles_comunes

grafico_starters = bp.c_grafico(starters, "Entrantes", "red")
grafico_mains = bp.c_grafico(mains, "Principales", "blue")
grafico_pizzas = bp.c_grafico(pizzas, "Pizzas","green")
grafico_desserts = bp.c_grafico(desserts, "Postres", "orange")
grafico_cocktails = bp.c_grafico(cocktails, "Cocteles", "purple")


fig = go.Figure()


fig.add_traces([grafico_starters, grafico_mains, grafico_pizzas, grafico_desserts, grafico_cocktails])

fig.update_traces(visible=False)
fig.data[0].visible = True  


buttons = [
    dict(label="Entrantes", method="update", args=[{"visible": [True, False, False, False, False]}, {"title": "Entrantes mas comunes"}]),
    dict(label="Principales", method="update", args=[{"visible": [False, True, False, False, False]}, {"title": "Principales mas comunes"}]),
    dict(label="Pizzas", method="update", args=[{"visible": [False, False, True, False, False]}, {"title": "Pizzas mas comunes"}]),
    dict(label="Postres", method="update", args=[{"visible": [False, False, False, True, False]}, {"title": "Postres mas comunes"}]),
    dict(label="Cocteles", method="update", args=[{"visible": [False, False, False, False, True]}, {"title": "Cocteles mas comunes"}]),]
   
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
        )
    ]
)


pio.show(fig)



In [6]:
#Grafico de pastel sobre el porciento de plantas por municipio :)
info_plantas = data2[data2["electric generator"].notna()]
total_generadores = info_plantas["electric generator"].sum()
generadores_por_municipio = info_plantas.groupby("location.municipe")["electric generator"].sum()
porcentaje_por_municipio = (generadores_por_municipio / total_generadores * 100).round(2)
porcentaje_por_municipio.index = [municipio.strip().lower() for municipio in porcentaje_por_municipio.index]


fig = px.pie(
    names=porcentaje_por_municipio.index,
    values=porcentaje_por_municipio.values,
    title="Porcentaje del Total de Generadores Electricos por Municipio"
)

fig.update_traces(textinfo="percent+label")
fig.show()

In [7]:
#Tabla donde da la clasificacion promedio de los restaurantes por municipio :)
data2["location.municipe"] = data2["location.municipe"].str.strip().str.lower()

calificacion_promedio_por_municipio = data2.groupby("location.municipe")["qualification"].mean().round(2)
tabla_calificaciones = calificacion_promedio_por_municipio.reset_index()
tabla_calificaciones.columns = ["municipio", "calificacion promedio"]
tabla_calificaciones = tabla_calificaciones.sort_values(by="calificacion promedio", ascending=False)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(tabla_calificaciones.columns),
                fill_color="pink",
                align="left"),
    cells=dict(values=[tabla_calificaciones.municipio, tabla_calificaciones["calificacion promedio"]],
               fill_color="lavender",
               align="left"))
])

fig.update_layout(title="Calificacion Promedio de Restaurantes por Municipio segun Google")
fig.show()




In [8]:
#Tabla que muestra el top 5 de restaurantes de cada municipio segun la clasificacion:)
data2 = data2.dropna(subset=["qualification"])
ranking = data2.sort_values(by=["location.municipe", "qualification"], ascending=[True, False])

tabla_ranking = ranking.groupby("location.municipe").head(5).reset_index(drop=True)

r_municipios = tabla_ranking["location.municipe"].unique()
r_info_municipio = {municipio: tabla_ranking[tabla_ranking["location.municipe"] == municipio] for municipio in r_municipios}

def mostrar_tabla(municipio):
    display(r_info_municipio[municipio][["name", "qualification"]])

r_selector_municipio = wi.Dropdown(options=r_municipios, description="Municipio:")
wi.interact(mostrar_tabla, municipio=r_selector_municipio)

interactive(children=(Dropdown(description='Municipio:', options=('10 de octubre', 'arroyo naranjo', 'boyeros'…

<function __main__.mostrar_tabla(municipio)>

In [9]:
#Grafica que muestra la capacidad promedio de los restaurantes que aceptan reservacion :)
restaurantes_con_reserva = data2[data2["reservation"] == True]
capacidad_promedio = restaurantes_con_reserva["capacity"].mean()

fig = px.bar(
    x=["Restaurantes con reserva"],
    y=[capacidad_promedio],
    labels={"x": "Categoría", "y": "Capacidad Promedio"},
    title="Capacidad Promedio de Restaurantes que Aceptan Reserva",
    text=[f"{capacidad_promedio:.0f}"]
)

fig.update_traces(textposition="outside")
fig.show()





In [11]:

#Mapa con los restaurantes ubicado :)
mapa = fl.Map(location=[23.1, -82.4], zoom_start=11)

lahabanamapa = {"type": "FeatureCollection", "features": [{"geometry": {"type": "Polygon", "coordinates": [[[-82.5403, 23.0517], [-82.5289, 23.0628], [-82.5239, 23.0735], [-82.509, 23.0829], [-82.5045, 23.0806], [-82.4981, 23.0844], [-82.5026, 23.0865], [-82.4957, 23.0937], [-82.4911, 23.0937], [-82.4894, 23.0897], [-82.4872, 23.0951], [-82.4835, 23.0947], [-82.4781, 23.0901], [-82.4783, 23.0949], [-82.4606, 23.0936], [-82.4609, 23.0976], [-82.4554, 23.0961], [-82.4535, 23.1017], [-82.4346, 23.1193], [-82.4152, 23.1334], [-82.4121, 23.1339], [-82.4119, 23.1305], [-82.4053, 23.1406], [-82.3912, 23.1478], [-82.3696, 23.1424], [-82.3579, 23.1468], [-82.3522, 23.145], [-82.345, 23.1353], [-82.3478, 23.1286], [-82.3577, 23.122], [-82.351, 23.1205], [-82.3448, 23.1266], [-82.3422, 23.1147], [-82.3343, 23.1123], [-82.3406, 23.1243], [-82.3359, 23.1273], [-82.3374, 23.1347], [-82.328, 23.1303], [-82.3281, 23.1382], [-82.3229, 23.1427], [-82.3446, 23.1424], [-82.3563, 23.1516], [-82.3244, 23.1664], [-82.3026, 23.1715], [-82.2944, 23.1688], [-82.2911, 23.1637], [-82.2892, 23.1702], [-82.2772, 23.1754], [-82.2442, 23.1786], [-82.2402, 23.1732], [-82.2393, 23.1783], [-82.228, 23.1806], [-82.214, 23.1785], [-82.2115, 23.1724], [-82.2121, 23.1785], [-82.2045, 23.1806], [-82.1518, 23.172], [-82.1258, 23.1744], [-82.1225, 23.1684], [-82.1161, 23.1715], [-82.1209, 23.1724], [-82.1185, 23.1737], [-82.098, 23.1729], [-82.0929, 23.1761], [-82.0925, 23.168], [-82.1016, 23.1666], [-82.1025, 23.1598], [-82.082, 23.1487], [-82.0799, 23.1245], [-82.0931, 23.1221], [-82.0861, 23.1055], [-82.1016, 23.0931], [-82.1015, 23.0847], [-82.1087, 23.0806], [-82.1058, 23.0767], [-82.1096, 23.0683], [-82.1011, 23.0555], [-82.1148, 23.0547], [-82.1445, 23.0685], [-82.15, 23.075], [-82.1562, 23.0744], [-82.1764, 23.0576], [-82.2231, 23.064], [-82.2256, 23.0165], [-82.1964, 22.9972], [-82.1979, 22.993], [-82.2115, 22.9944], [-82.2167, 22.9997], [-82.2283, 22.9804], [-82.2618, 22.9734], [-82.2475, 22.9564], [-82.2676, 22.9578], [-82.2798, 22.9519], [-82.2976, 22.9541], [-82.3009, 22.9627], [-82.3056, 22.9648], [-82.3132, 22.9649], [-82.3304, 22.9563], [-82.3648, 22.9667], [-82.3775, 22.9657], [-82.3759, 22.9558], [-82.3658, 22.9405], [-82.3868, 22.9337], [-82.4152, 22.9483], [-82.4287, 22.9466], [-82.4336, 22.9403], [-82.4458, 22.9471], [-82.4436, 22.9814], [-82.4482, 22.9851], [-82.4578, 22.9775], [-82.4661, 22.982], [-82.4672, 22.9783], [-82.4843, 22.9729], [-82.498, 22.9947], [-82.5012, 23.0095], [-82.5081, 23.0153], [-82.5066, 23.0211], [-82.4982, 23.0262], [-82.4949, 23.0323], [-82.4973, 23.0392], [-82.4917, 23.0411], [-82.4889, 23.0523], [-82.4921, 23.0736], [-82.4972, 23.0736], [-82.5021, 23.0687], [-82.5108, 23.0703], [-82.5329, 23.0523], [-82.5403, 23.0517]]]}, "type": "Feature", "properties": {"province": "La Habana", "country": "Cuba", "region": "province", "country_domain": "cu", "province_id": "lha", "DPA_province_code": "23"}}]}
fl.GeoJson(lahabanamapa).add_to(mapa)
for i, row in data2.iterrows():
    coordenadas = [float(coord) for coord in row["location.coordinates"].split(", ")]
    nombre = row["name"]
    direccion = row["location.address"]
    especializacion = row["specialized_food"]
   

    fl.Marker(
        location=coordenadas,
        popup=(
            f"<b style='font-size: 18px; '>{nombre}</b><br>"
            f"Direccion: {direccion}",
            f"Especialidades: {especializacion}",
          ),
          icon=fl.Icon(icon="cutlery", prefix="fa", color="red")
    ).add_to(mapa)
display(mapa)

